In [2]:
!pip install pandas
!pip install sumy

In [3]:
import pandas as pd
from pandas import DataFrame as df
from pyspark.sql.functions import explode
from pyspark.sql.functions import explode, trim, regexp_extract, array_join, col, regexp_replace, concat_ws, split

# 실수 형태만 추출하는 함수 정의
def extract_float(text):
    import re
    result = re.findall(r'\d+\.\d+', text)
    if result:
        return float(result[0])
    else:
        return None

# extract_float UDF(사용자 정의 함수) 생성
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

extract_float_udf = udf(extract_float, FloatType())

from pyspark.sql.functions import lower, col

In [37]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType

In [58]:
# 데이터베이스에서 데이터 읽기
df = spark.read.jdbc(url=jdbc_url, table="COURSE", properties=connection_properties)

# 데이터 확인
df.show()

+---------+--------------+---+-------+--------------+-----------+----+---+----------+-----------+---------+---------+----------+----------+
|course_id|subcategory_id|url|summary|num_of_lecture|num_of_stud|rate|img|instructor|kor_support|reg_price|dis_price|updated_at|created_at|
+---------+--------------+---+-------+--------------+-----------+----+---+----------+-----------+---------+---------+----------+----------+
+---------+--------------+---+-------+--------------+-----------+----+---+----------+-----------+---------+---------+----------+----------+



In [6]:
# 주어진 데이터
data = [
    {"category_name": "대분류1", "subcategory_name": "소분류1", "course_title": "강의1", "summary": "요약1", "url": "url1", 
     "img": "img1", "new_course_img": "new_img1", "reg_price": 10000, "dis_price": 8000, 
     "tags": ["tag1", "tag2"], "section_num": 1, "section_name": "Section1", "num_of_section": 3, 
     "subsection_num": 1, "subsection_name": "Subsection1", "num_of_subsection": 5},
    {"category_name": "대분류2", "subcategory_name": "소분류2", "course_title": "강의2", "summary": "요약2", "url": "url2", 
     "img": "img2", "new_course_img": "new_img2", "reg_price": 15000, "dis_price": 12000, 
     "tags": ["tag3", "tag4"], "section_num": 2, "section_name": "Section2", "num_of_section": 4, 
     "subsection_num": 2, "subsection_name": "Subsection2", "num_of_subsection": 6}
]

# 스키마 정의
schema = StructType([
    StructField("category_name", StringType(), True),
    StructField("subcategory_name", StringType(), True),
    StructField("course_title", StringType(), True),
    StructField("summary", StringType(), True),
    StructField("url", StringType(), True),
    StructField("img", StringType(), True),
    StructField("new_course_img", StringType(), True),
    StructField("reg_price", IntegerType(), True),
    StructField("dis_price", IntegerType(), True),
    StructField("tags", ArrayType(StringType()), True),
    StructField("section_num", IntegerType(), True),
    StructField("section_name", StringType(), True),
    StructField("num_of_section", IntegerType(), True),
    StructField("subsection_num", IntegerType(), True),
    StructField("subsection_name", StringType(), True),
    StructField("num_of_subsection", IntegerType(), True)
])


# DataFrame 생성
df = spark.createDataFrame(data, schema)

# DataFrame 출력
df.show(truncate=False)

spark.stop()

+-------------+----------------+------------+-------+----+----+--------------+---------+---------+------------+-----------+------------+--------------+--------------+---------------+-----------------+
|category_name|subcategory_name|course_title|summary|url |img |new_course_img|reg_price|dis_price|tags        |section_num|section_name|num_of_section|subsection_num|subsection_name|num_of_subsection|
+-------------+----------------+------------+-------+----+----+--------------+---------+---------+------------+-----------+------------+--------------+--------------+---------------+-----------------+
|대분류1      |소분류1         |강의1       |요약1  |url1|img1|new_img1      |10000    |8000     |[tag1, tag2]|1          |Section1    |3             |1             |Subsection1    |5                |
|대분류2      |소분류2         |강의2       |요약2  |url2|img2|new_img2      |15000    |12000    |[tag3, tag4]|2          |Section2    |4             |2             |Subsection2    |6                |
+-------------+

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [31]:
import re
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_extract, regexp_replace, explode, udf
from pyspark.sql.types import ArrayType, StringType

# SparkSession 초기화
spark = SparkSession.builder \
    .appName("DataFrame Creation") \
    .getOrCreate()

# JSON 파일을 pandas DataFrame으로 읽기
pandas_df = pd.read_json("codeit_roadmaps_sample.json")

# pandas DataFrame을 Spark DataFrame으로 변환
spark_df = spark.createDataFrame(pandas_df)

# explode를 사용하여 roadmap 배열을 행으로 변환
exploded_df = spark_df.withColumn("roadmap", explode(col("roadmap")))

# 필요한 컬럼 선택 및 이름 변경
codeit_df = exploded_df.select(
    col("roadmap_categ"),
    col("roadmap.title").alias("title"),
    col("roadmap.lecture_num").alias("lecture_num"),
    col("roadmap.summary").alias("summary"),
    col("roadmap.star").alias("star")
)

def extract_float(star):
    if star is None:
        return None
    match = re.search(r"(\d+\.\d+)", star)
    return float(match.group(1)) if match else None

extract_float_udf = udf(extract_float, StringType())

# 각 컬럼의 정제 및 이름 변경
codeit_df = codeit_df.withColumn("num_of_lecture", regexp_extract(col("lecture_num"), r"\d+", 0)) \
                     .withColumn("summary", regexp_replace(col("summary"), "\n", "")) \
                     .withColumn("rate", extract_float_udf("star")) \
                     .select("roadmap_categ", "title", "num_of_lecture", "summary", "rate")

# 결과 출력
codeit_df.show(truncate=False)

# 각 컬럼 출력
# codeit_df.select("title").show(truncate=False)
# codeit_df.select("num_of_lecture").show(truncate=False)
# codeit_df.select("summary").show(truncate=False)
# codeit_df.select("rate").show(truncate=False)

# SparkSession 종료
spark.stop()


+---------------+-------------------------------+--------------+--------------------------------------------------+----+
|roadmap_categ  |title                          |num_of_lecture|summary                                           |rate|
+---------------+-------------------------------+--------------+--------------------------------------------------+----+
|커리어         |데이터 사이언티스트            |25            |데이터 사이언티스트 취업의가장 빠른 길            |4.9 |
|커리어         |Python 풀스택 개발자           |37            |혼자서 다 해내는풀스택 개발자가 되는 가장 빠른 길 |4.9 |
|커리어         |JavaScript 프론트엔드 개발자   |33            |프론트엔드 개발자 취업의가장 빠른 길              |4.9 |
|커리어         |null                           |null          |null                                              |null|
|기초 강의      |null                           |null          |null                                              |null|
|기초 강의      |HTML/CSS로 웹사이트 만들기     |6             |웹 사이트를 만들 줄 아는인재가 되는 빠른 길       |4.9 |
|기초 강의      |Python 기초            

In [21]:
import re
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, udf
from pyspark.sql.types import ArrayType, StructType, StructField, StringType

# SparkSession 초기화
spark = SparkSession.builder \
    .appName("Section Extraction") \
    .getOrCreate()

# JSON 파일을 pandas DataFrame으로 읽기
pandas_df = pd.read_json("codeit_roadmaps_sample.json")

# pandas DataFrame을 Spark DataFrame으로 변환
spark_df = spark.createDataFrame(pandas_df)

# explode를 사용하여 roadmap 배열을 행으로 변환
exploded_df = spark_df.withColumn("roadmap", explode(col("roadmap")))

# 필요한 컬럼 선택 및 이름 변경
codeit_df = exploded_df.select(
    col("roadmap_categ"),
    col("roadmap.title").alias("title"),
    col("roadmap.section").alias("section")
)

# 섹션 정보를 나누기 위한 UDF 생성
@udf(returnType=ArrayType(StructType([
    StructField("section_num", StringType(), True),
    StructField("section_name", StringType(), True)
])))
def split_sections(section_str):
    if not isinstance(section_str, str):
        return []
    pattern = r"STEP\n(\d+)\n([^\n]+)"
    matches = re.findall(pattern, section_str)
    sections = [{"section_num": int(num), "section_name": name} for num, name in matches]
    return sections

# 섹션 정보를 배열로 변환
sections_df = codeit_df.withColumn("sections", split_sections(col("section")))

# 각 섹션 정보를 개별 행으로 분리
sections_exploded_df = sections_df.withColumn("sections", explode(col("sections"))) \
                                  .select(col("roadmap_categ"),
                                          col("title"),
                                          col("sections.section_num"),
                                          col("sections.section_name"))

# 결과 출력
sections_exploded_df.show(truncate=False)

# 결과를 pandas DataFrame으로 변환 (선택사항)
result_df = sections_exploded_df.toPandas()

# # 결과를 CSV 파일로 저장 (선택사항)
# result_df.to_csv("section_info.csv", index=False)

# SparkSession 종료
spark.stop()

+-------------+--------------------+-----------+--------------------------+
|roadmap_categ|title               |section_num|section_name              |
+-------------+--------------------+-----------+--------------------------+
|커리어       |데이터 사이언티스트 |1          |프로그래밍 커리어 살펴보기|
|커리어       |데이터 사이언티스트 |2          |프로그래밍 기본기         |
|커리어       |데이터 사이언티스트 |3          |데이터 사이언스 기초      |
|커리어       |데이터 사이언티스트 |4          |데이터 분석 프로젝트      |
|커리어       |데이터 사이언티스트 |5          |데이터 분석 실전          |
|커리어       |데이터 사이언티스트 |6          |머신 러닝                 |
|커리어       |데이터 사이언티스트 |7          |딥 러닝                   |
|커리어       |데이터 사이언티스트 |8          |협업                      |
|커리어       |Python 풀스택 개발자|1          |프로그래밍 커리어 살펴보기|
|커리어       |Python 풀스택 개발자|2          |프로그래밍 기초           |
|커리어       |Python 풀스택 개발자|3          |웹 개발 시작하기          |
|커리어       |Python 풀스택 개발자|4          |웹 퍼블리싱               |
|커리어       |Python 풀스택 개발자|5          |CLI 환경 기본기           |
|커리어       |Pyt

In [36]:
import pandas as pd
import re
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
import nltk

# NLTK 데이터 다운로드
nltk.download('punkt')

def remove_emojis_and_special_chars(text):
    emoji_pattern = re.compile(
        "[" 
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002500-\U00002BEF"  # chinese char
        "\U00002702-\U000027B0"
        "\U00002702-\U000027B0"
        # "\U0001f926-\U0001f937"
        "\U00010000-\U0010ffff"
        "\u2640-\u2642"
        "\u2600-\u2B55"
        "\u200d"
        "\u23cf"
        "\u23e9"
        "\u231a"
        "\ufe0f"  # dingbats
        "\u3030"
        "]+", 
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

# 리뷰를 요약하는 함수 정의
def summarize_review(review_text):
    parser = PlaintextParser.from_string(review_text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, 1)  # 리뷰를 한 문장으로 요약
    if len(summary) > 0:
        summary_text = str(summary[0])
        if summary_text != "No summary available." and len(summary_text) > 15:
            return summary_text
    return None

# JSON 파일을 pandas DataFrame으로 읽기
pandas_df = pd.read_json("codeit_roadmaps_sample.json")

# 데이터프레임 확장 함수 정의
def extract_reviews(row):
    review_summaries = []
    roadmap_categ = row['roadmap_categ']
    for roadmap_detail in row['roadmap']:
        if roadmap_detail is not None:
            title = roadmap_detail.get('title', '')
            review_text = roadmap_detail.get('review', '')
            review_lines = review_text.split('\n')
            reviews = [review_lines[i] for i in range(2, len(review_lines), 3)]  # 리뷰 텍스트만 추출
            for review in reviews:
                if review:
                    clean_review = remove_emojis_and_special_chars(review)
                    summary = summarize_review(clean_review)
                    if summary is not None:
                        review_summaries.append((roadmap_categ, title, summary))
    return review_summaries

# 리뷰 정보를 추출하여 새로운 열에 저장
pandas_df['review_summary'] = pandas_df.apply(extract_reviews, axis=1)

# 새로운 DataFrame 생성
reviews_df = pd.DataFrame(pandas_df['review_summary'].explode().tolist(), columns=['Category', 'Title', 'Review Summary'])

# 각 카테고리와 타이틀별로 리뷰 요약을 그룹화
grouped_reviews = reviews_df.groupby(['Category', 'Title'])['Review Summary'].apply(list).reset_index()

# # 결과 출력
# for index, row in grouped_reviews.iterrows():
#     category = row['Category']
#     title = row['Title']
#     reviews = row['Review Summary'][:3]  # 첫 3개의 요약만 추출
#     print(f"Category: {category}, Title: {title}")
#     for i, review in enumerate(reviews, 1):
#         print(f"Review Summary {i}: {review}")
#     print()

import random

# 결과 출력
for index, row in grouped_reviews.iterrows():
    category = row['Category']
    title = row['Title']
    reviews = row['Review Summary']
    random_reviews = random.sample(reviews, min(3, len(reviews)))  # 최대 3개의 요약을 랜덤하게 선택
    print(f"Category: {category}, Title: {title}")
    for i, review in enumerate(random_reviews, 1):
        print(f"Review Summary {i}: {review}")
    print()


Category: 기초 강의, Title: Figma로 시작하는 UI 디자인
Review Summary 1: 포토샵, illu의 경우 비용과 복잡성등으로 접근성이 떨어져 디자인이 막연히 어렵다고 느꼈었는데요.
Review Summary 2: 실습자료를 꼼꼼히 준비해주셔서 예시를 통해 이론을 적용해보는 데에 유용했습니다:)
Review Summary 3: 예제 파일이 제공되기 때문에 강의를 들으면서 하나씩 따라해보면 훨씬 더 쉽고 빠르게 이해할 수 있어요!

Category: 기초 강의, Title: HTML/CSS로 웹사이트 만들기
Review Summary 1: 반응형 웹이 어떻게 만들어지는지 항상 궁금했었는데 잘 배우고 갑니다!
Review Summary 2: 너무 잼있게 이해 잘되게 잘 설명해주심!
Review Summary 3: 시키는대로 따라하다보니 토픽 하나 끝냈네요!

Category: 기초 강의, Title: JavaScript 기초
Review Summary 1: 수업도 훌륭하고 무엇보다 퀴즈나 실습과제 후 첨부된 해설이 초보자 입장에서 친절하고 디테일도 놓지지 않고 챙겨줘서 공부에 도움이 많이 됩니다
Review Summary 2: 강의 자체가 가독성이 좋아서 빠르게 들을수 있었습니다!
Review Summary 3: 알기 쉽게 기초 문법도 알려주시고 실습도 따라하면서 체득하기 좋아요

Category: 기초 강의, Title: Python 기초
Review Summary 1: 코딩입문자입니다 진도 차근히 나가면서 도움받고 있습니다.
Review Summary 2: 적당한 난이도의 예제로 복습하기 좋았다
Review Summary 3: 파이썬을 응용할 수 있게 되어 좋았다.

Category: 기초 강의, Title: 비개발자를 위한 SQL 데이터 분석
Review Summary 1: 혼자 sql에 관해서 공부하면서도 제일 어려웠던 건 전체적으로 데이터분석에 접근하는 방법이 이해가 안되고 막막했었는데, 이 강의를 들으면서 개념이 서고 전체적인

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

# PySpark 세션 생성
spark = SparkSession.builder \
    .appName("PostgreSQL Connection Example") \
    .getOrCreate()

# 데이터프레임 생성
data = [(6, "데이터 사이언티스트", "temp url2", "temp summary", None, None, None, None, "temp instructor", 15000, 10000)]

columns = ["subcategory_id", "course_title", "url", "summary", "num_of_lecture", "num_of_stud", "rate", "img", "instructor", "reg_price", "dis_price"]
# df = spark.createDataFrame(data, columns)

# 새로운 데이터프레임 스키마 생성
schema = StructType([
    StructField("subcategory_id", IntegerType(), True),
    StructField("course_title", StringType(), True),
    StructField("url", StringType(), True),
    StructField("summary", StringType(), True),
    StructField("num_of_lecture", IntegerType(), True),
    StructField("num_of_stud", IntegerType(), True),
    StructField("rate", IntegerType(), True),
    StructField("img", StringType(), True),
    StructField("instructor", StringType(), True),
    StructField("reg_price", IntegerType(), True),
    StructField("dis_price", IntegerType(), True)
])

# 데이터프레임의 스키마 변경
df = spark.createDataFrame(data, schema)

# PostgreSQL 연결 설정
jdbc_url = "jdbc:postgresql://54.180.100.57:5432/lecture_db"
connection_properties = {
    "user": "khudade",
    "password": "khudade",
    "driver": "org.postgresql.Driver"
}

# 데이터프레임을 PostgreSQL에 삽입
df.write.jdbc(url=jdbc_url, table="COURSE", mode="append", properties=connection_properties)